In [1]:
import pandas as pd
import numpy as np
from Bio import Entrez, Medline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor
import time
import re

# Téléchargement des stopwords pour le traitement
nltk.download('stopwords')


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/legbedje/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
Entrez.email = "votre-email@example.com"  # Remplacez par votre email
Entrez.api_key = "votre-clé-api"          # Remplacez par votre clé API


In [ ]:
# Récupérer des articles en lots avec gestion des erreurs
def fetch_articles_batch(id_batch, retries=3):
    ids_str = ",".join(id_batch)
    for attempt in range(retries):
        try:
            handle = Entrez.efetch(db="pubmed", id=ids_str, rettype="medline", retmode="text")
            records = Medline.parse(handle)
            records_list = list(records)
            handle.close()
            return records_list
        except Exception as e:
            print(f"Erreur pour le lot, tentative {attempt + 1}/{retries}: {e}")
            time.sleep(1)  # Pause avant une nouvelle tentative
    print(f"Échec après {retries} tentatives pour le lot : {id_batch}")
    return []

# Paralléliser la récupération des articles
def fetch_pubmed_articles(pubmed_ids, batch_size=100, max_workers=5):
    articles = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_articles_batch, pubmed_ids[i:i + batch_size])
                   for i in range(0, len(pubmed_ids), batch_size)]
        for future in futures:
            articles.extend(future.result())
            time.sleep(0.1)  # Pause légère pour éviter la surcharge API
    return articles


In [ ]:
# Fonction de nettoyage des textes
def clean_text(text):
    text = text.lower()  # Mise en minuscule
    text = re.sub(r'\[.*?\]', '', text)  # Suppression des références
    text = re.sub(r'\W', ' ', text)  # Suppression des caractères spéciaux
    text = re.sub(r'\s+', ' ', text)  # Suppression des espaces multiples
    text = text.strip()  # Suppression des espaces en début/fin
    return text

# Nettoyer les abstracts
def preprocess_abstracts(df):
    stop_words = set(stopwords.words('english'))
    df['cleaned_abstract'] = df['Abstract'].apply(lambda x: ' '.join(
        [word for word in clean_text(x).split() if word not in stop_words]))
    return df


In [ ]:
def descriptive_statistics(df):
    # Ajouter des colonnes pour analyse
    df['abstract_length'] = df['Abstract'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
    df['num_authors'] = df['Authors'].apply(lambda x: len(x.split(',')) if isinstance(x, str) else 0)

    # Statistiques descriptives
    desc_stats = df[['abstract_length', 'num_authors']].describe()
    print("Statistiques descriptives :\n", desc_stats)

    # Visualisation
    plt.figure(figsize=(10, 6))
    sns.histplot(df['abstract_length'], bins=50, kde=True)
    plt.title("Distribution de la longueur des résumés")
    plt.xlabel("Nombre de mots")
    plt.ylabel("Fréquence")
    plt.show()

    plt.figure(figsize=(10, 6))
    sns.histplot(df['num_authors'], bins=50, kde=False)
    plt.title("Distribution du nombre d'auteurs par article")
    plt.xlabel("Nombre d'auteurs")
    plt.ylabel("Fréquence")
    plt.show()


In [ ]:
def vectorize_and_cluster(df):
    # Vectorisation avec TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_features=1000)
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_abstract'])

    # Réduction de dimensionnalité avec PCA
    pca = PCA(n_components=2)
    tfidf_reduced = pca.fit_transform(tfidf_matrix.toarray())

    # Clustering avec K-means
    kmeans = KMeans(n_clusters=5, random_state=0)
    clusters = kmeans.fit_predict(tfidf_reduced)

    # Ajouter les clusters au DataFrame
    df['cluster'] = clusters

    # Visualisation des clusters
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=tfidf_reduced[:, 0], y=tfidf_reduced[:, 1], hue=df['cluster'], palette='viridis', alpha=0.7)
    plt.title("Visualisation des clusters des articles (PCA)")
    plt.xlabel("Composante principale 1")
    plt.ylabel("Composante principale 2")
    plt.show()
    
    return tfidf_matrix, clusters


In [ ]:
if __name__ == "__main__":
    # 1. Charger les IDs PubMed (exemple depuis un fichier CSV)
    pubmed_ids_csv = pd.read_csv("pubmed_ids.csv")  # Remplacez par votre fichier
    pubmed_ids = pubmed_ids_csv['pubmedid'].astype(str).tolist()

    # 2. Récupérer les articles
    start_time = time.time()
    pubmed_data = fetch_pubmed_articles(pubmed_ids, batch_size=100, max_workers=5)
    end_time = time.time()
    print(f"Extraction terminée en {end_time - start_time:.2f} secondes")

    # 3. Extraire les champs pertinents
    extracted_data = []
    for record in pubmed_data:
        pmid = record.get('PMID', '')
        title = record.get('TI', '')
        abstract = record.get('AB', '')
        date = record.get('DP', '')
        authors = ", ".join(record.get('AU', []))
        
        extracted_data.append({
            'PMID': pmid,
            'Title': title,
            'Abstract': abstract,
            'Date': date,
            'Authors': authors
        })

    # 4. Créer un DataFrame
    df = pd.DataFrame(extracted_data)

    # 5. Nettoyer les données
    df = preprocess_abstracts(df)

    # 6. Analyse exploratoire
    descriptive_statistics(df)

    # 7. Vectorisation et clustering
    tfidf_matrix, clusters = vectorize_and_cluster(df)

    # 8. Sauvegarder les résultats
    df.to_csv("pubmed_clustered_data.csv", index=False)# fichier lourd je ne peut pas le commit
    print("Données sauvegardées dans pubmed_clustered_data.csv")
